## DeepFool Algorithm

Cálculo dos Gradientes

In [ ]:
import tensorflow as tf
import numpy as np
from copy import deepcopy

def get_gradients(model, x): # get gradients for all the classes at the same time (with respect to the input)

    with tf.GradientTape() as gtape:
        inputs = [tf.cast(input_value, dtype = tf.float64) for input_value in x]
        for input_value in inputs:
            gtape.watch(input_value)
        results = model(inputs)

    gradients = gtape.gradient(results, inputs)
    del tape
    return [grad.numpy() for grad in gradients], results.numpy()

In [ ]:
def deepfool(model, x0, eta=0.02, max_iter=20, epsilon=1e-6):
    """
    Improved DeepFool algorithm with gradient optimization, stopping criteria, and dynamic overshoot.
    """
    # Initial label and model output
    f_x0 = model(x0).numpy().flatten()
    label_x0 = np.argmax(f_x0)
    
    xi = deepcopy(x0)
    label_xi = label_x0
    loop_i = 0
    perturbations = []

    while label_xi == label_x0 and loop_i < max_iter:
        grad_f_xi, f_xi = get_gradients(model, xi)  # Gradients and logits for xi
        grad_f_label_x0 = [g[label_x0] for g in grad_f_xi]  # Gradient for true class
        
        min_fk_wk = np.inf  # Initialize minimum distance to decision boundary
        w_l, f_l = None, None

        # Loop through other classes
        for k in range(f_xi.shape[1]):  # Iterate over all classes
            if k == label_x0:
                continue
            grad_f_k = [g[k] for g in grad_f_xi]  # Gradient for class k
            w_k = [g_k - g_label for g_k, g_label in zip(grad_f_k, grad_f_label_x0)]  # Direction
            w_k_norm = np.sqrt(np.sum([np.linalg.norm(w)**2 for w in w_k]))
            f_k = f_xi[0, k] - f_xi[0, label_x0]  # Difference in logits
            fk_wk = abs(f_k) / (w_k_norm + epsilon)  # Distance to boundary

            if fk_wk < min_fk_wk:
                min_fk_wk = fk_wk
                w_l, f_l = w_k, f_k

        # Compute perturbation step
        w_l_squared_norm = np.sum([np.linalg.norm(w)**2 for w in w_l])
        ri_const = abs(f_l) / (w_l_squared_norm + epsilon)
        ri = [ri_const * w for w in w_l]  # Perturbation
        perturbations.append(ri)

        # Update perturbed input
        xi = [xi_item + (1 + eta) * ri_item for xi_item, ri_item in zip(xi, ri)]
        xi = [tf.clip_by_value(xi_item, 0, 1).numpy() for xi_item in xi]  # Ensure input bounds [0,1]
        
        # Update label and loop count
        label_xi = np.argmax(model(xi).numpy().flatten())
        loop_i += 1

        # Stopping criteria: small perturbation
        if np.sqrt(np.sum([np.linalg.norm(r)**2 for r in ri])) < epsilon:
            break

    # Summing up all perturbations
    total_perturbation = [np.sum([r[i] for r in perturbations], axis=0) for i in range(len(x0))]

    return total_perturbation, loop_i, label_xi

In [ ]:
def example_robustness(x, r):
    """
    Calculate robustness measure for an individual example.
    """
    r_norm = np.sqrt(np.sum([np.linalg.norm(r_input)**2 for r_input in r]))
    x_norm = np.sqrt(np.sum([np.linalg.norm(x_input)**2 for x_input in x]))
    return r_norm / x_norm


def model_robustness(example_robustness_list):
    """
    Calculate mean and standard deviation of robustness for the model.
    """
    mean = np.mean(example_robustness_list)
    std = np.std(example_robustness_list)
    return mean, std